In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ranking import *
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [3]:
df = pd.read_csv("res\\movie\\rating.csv")

In [4]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [5]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [6]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [7]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


In [8]:
hits: HITS = HITSFactory().read_pd(data=data, source='userId', target='movieId', edge_attr='rating', personalization=1)

C:\Users\odo20\AppData\Roaming\Python\Python312\site-packages\scipy\sparse\_base.py:713: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


In [9]:
prediction_movie_for_user1 = hits.get_rank()

In [10]:
print(prediction_movie_for_user1.sort_values(by='hubness', ascending=False).head())

       target  authoritativeness  hubness
0        2288       2.207916e-04      0.0
17760   97709       1.086575e-07      0.0
17836  107408       3.046013e-08      0.0
17835  104780       7.874532e-08      0.0
17834  103593       2.398938e-08      0.0


In [11]:
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~(prediction_movie_for_user1['target'].isin(train1['movieId']))]
print(len(prediction_movie_for_user1_without_famous))
prediction_movie_for_user1_without_famous.head()

26657


,target,authoritativeness,hubness
87,3,0.001288,0.0
88,62,0.002852,0.0
89,70,0.000462,0.0
90,110,0.010035,0.0
91,242,0.000065,0.0


In [12]:
pred = prediction_movie_for_user1_without_famous.sort_values(by='authoritativeness', ascending=False).head(len(test1))
print(len(pred))
pred.head()

88


,target,authoritativeness,hubness
592,296,0.013617,0.0
153,318,0.013275,0.0
305,356,0.012501,0.0
329,150,0.012210,0.0
157,457,0.012003,0.0


In [13]:
def accuracy_without_order(true_labels, predicted_labels):
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set)
    
    return accuracy

In [14]:
print(accuracy_without_order(test1['movieId'], pred['target']))

0.14772727272727273


In [15]:
movie_df = pd.read_csv('res\\movie\\movie.csv')

In [16]:
prediction_moviename = pd.merge(movie_df, pred, left_on='movieId', right_on='target', how='inner')
prediction_moviename.sort_values(by='authoritativeness', ascending=False).head()

,movieId,title,genres,target,authoritativeness,hubness
34,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,296,0.013617,0.0
38,318,"Shawshank Redemption, The (1994)",Crime|Drama,318,0.013275,0.0
44,356,Forrest Gump (1994),Comedy|Drama|Romance|War,356,0.012501,0.0
19,150,Apollo 13 (1995),Adventure|Drama|IMAX,150,0.012210,0.0
54,457,"Fugitive, The (1993)",Thriller,457,0.012003,0.0


In [17]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
print(len(test_moviename))
test_moviename.sort_values(by='rating', ascending=False).head()

88


,movieId,title,genres,userId,rating,timestamp
73,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,1,5.0,2005-04-02 23:30:19
87,8636,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,1,4.5,2005-04-02 23:44:53
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
54,3499,Misery (1990),Drama|Horror|Thriller,1,4.0,2005-04-02 23:35:18


In [18]:
test_moviename[test_moviename['movieId'].isin(prediction_moviename['target'])]

,movieId,title,genres,userId,rating,timestamp
0,2,Jumanji (1995),Adventure|Children|Fantasy,1,3.5,2005-04-02 23:53:47
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,1,3.5,2005-04-02 23:33:39
2,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,3.5,2005-04-02 23:29:40
4,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,1,4.0,2005-04-02 23:33:46
5,293,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller,1,4.0,2005-04-02 23:31:43
6,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1,4.0,2005-04-02 23:32:47
7,318,"Shawshank Redemption, The (1994)",Crime|Drama,1,4.0,2005-04-02 23:33:18
9,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,1,3.5,2005-04-02 23:45:57
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24


In [19]:

for_score = prediction_movie_for_user1.merge(test1, how='inner', left_on='target', right_on='movieId')
o = O(test1['rating'].mean())
t = T(prediction_movie_for_user1['authoritativeness'].mean())
i = I(t, o)
pairroid = Pairord(i)
pred = np.array(for_score['authoritativeness'])
true = np.array(for_score['rating'])
pred_map = np.vectorize(lambda x: t(x))
true_map = np.vectorize(lambda x: o(x))

pred_trasformed = pred_map(pred)
true_trasformed = true_map(true)

In [20]:
# Вычисляем метрики

accuracy = accuracy_score(true_trasformed, pred_trasformed)
precision = precision_score(true_trasformed, pred_trasformed)
recall = recall_score(true_trasformed, pred_trasformed)
f1 = f1_score(true_trasformed, pred_trasformed)

# Выводим результаты
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Pairorder:", pairroid(for_score['authoritativeness'], for_score['rating']))

Accuracy: 0.5681818181818182
Precision: 0.5492957746478874
Recall: 0.8666666666666667
F1 Score: 0.6724137931034483
Pairorder: 0.6712293388429752
